In [5]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install "transformers>=4.45.0" accelerate bitsandbytes huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.4 MB/s eta 0:00:0000:010:02
Note: you may need to restart the kernel to use updated packages.


Use the following code to check the login status with huggingface and llama.

Make sure:
1. The role is read, not fine-grained.
```bash
'role': 'read'
```
2. model is ok to process.
```bash
model ok, id: meta-llama/Llama-3.2-3B-Instruct
```

In [1]:
from huggingface_hub import whoami, model_info

# check whether you have been logged in
try:
    print("whoami:", whoami())
except Exception as e:
    print("whoami error:", e)

# check whether you have access to the model
try:
    info = model_info("meta-llama/Llama-3.2-3B-Instruct")
    print("model ok, id:", info.id)
except Exception as e:
    print("model_info error:", e)


/opt/anaconda3/envs/bias-detection/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


whoami: {'type': 'user', 'id': '691e14c9ad200e6e3b2e2c1d', 'name': 'juan27111', 'fullname': 'jingyuan liu', 'email': 'ljjjuan@bu.edu', 'emailVerified': True, 'canPay': False, 'billingMode': 'prepaid', 'periodEnd': 1767225600, 'isPro': False, 'avatarUrl': '/avatars/c72f708bdb7c43b1fb2d5c2f29a65201.svg', 'orgs': [{'type': 'org', 'id': '63864e015ea0bcbedb132e5a', 'name': 'BostonU', 'fullname': 'Boston University', 'email': None, 'canPay': False, 'billingMode': 'postpaid', 'periodEnd': None, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/1669746226855-63864a6a06858a85f592d5ae.png', 'roleInOrg': 'write', 'isEnterprise': False}], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'localllm', 'role': 'read', 'createdAt': '2025-12-05T01:49:42.167Z'}}}
model ok, id: meta-llama/Llama-3.2-3B-Instruct


Run the following code to load the model and access the local version. The first try would be a bit long, downloading models.

Remember to include your own standard of scoring into the evaluation prompt.

Parameters are also adjustable for specific needs.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="cpu"
)

if model.config.pad_token_id is None and tokenizer.eos_token_id is not None:
    model.config.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.52s/it]


In [5]:
def score_license(license_text: str) -> float:
    system_prompt = (
        "You are a strict security and risk assessment expert. "
        "Your ONLY job is to read the LICENSE text and output a single numeric score between 0 and 1. "
        "Do NOT explain, do NOT repeat the prompt, do NOT add any text."
    )

    user_prompt = f"""
Evaluate whether the following LICENSE text is sufficiently safe and detailed.

Criteria:
1) Safety safeguards against misuse (legal + ethical)
2) Detail about permissions, restrictions, and obligations
3) Explicit conditions preventing harmful applications

Scoring:
- 0 = not mentioning license related information at all
- 0.5 = not enough details on license
- 1 = fully safe and sufficiently detailed
- Output a SINGLE number between 0 and 1 (inclusive).
- Do NOT output anything except the number.

LICENSE:
{license_text}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,  
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=8,     
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

    score = float(answer)
    print(score)
    return score

In [6]:
example_text = 'Our model and weights are licensed for both researchers and commercial entities, upholding the principles of openness. Our mission is to empower individuals and industry, through this opportunity, while fostering an environment of discovery and ethical AI advancements.'
example_score = score_license(example_text)

/opt/anaconda3/envs/bias-detection/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/anaconda3/envs/bias-detection/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


0.25
